In [ ]:
# 한글이니까 konlpy 설치
!pip install konlpy

In [ ]:
# 긍정은 1이고 부정은 0이다.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
import tensorflow as tf

In [ ]:
# urllib.request.urlretrieve('https://raw.githubusercontent.com/pykwon/python/master/testdata_utf8/ratings_train.txt', filename='ratings_train.txt')
train_data=pd.read_table('https://raw.githubusercontent.com/pykwon/python/master/testdata_utf8/ratings_train.txt')
test_data=pd.read_table('https://raw.githubusercontent.com/pykwon/python/master/testdata_utf8/ratings_test.txt')
print(train_data[:2],train_data.shape) # (150000, 3)
print(test_data[:2], test_data.shape) # (50000, 3)

# data cleaning
print(train_data['document'].nunique(), test_data['document'].nunique()) #중복자료 있다.

train_data.drop_duplicates(subset=['document'], inplace=True)
print('샘플 수 : ', len(train_data))
print(train_data.groupby('label').size().reset_index(name='count')) #레이블별 건수 출력

print(train_data.isnull().values.any())
train_data=train_data.dropna(how='any')
print(train_data.shape)

# 한글 자료만 허용
train_data['document'] = train_data['document'].str.replace('[^가-힣 ]','')
print(train_data[:3])

# white space를 empty value로 변경(문단의 시작글자가 공백인것)
train_data['document'] = train_data['document'].str.replace('^ +', '') #문단의 시작글자가 공백일 경우 지우라는 뜻
train_data['document'].replace('',np.nan, inplace=True)
print(train_data.isnull().sum())
print(train_data.loc[train_data.document.isnull()][:3])

train_data=train_data.dropna(how='any')
print(train_data.shape) #(143028, 3)

test_data.drop_duplicates(subset=['document'],inplace=True)
test_data['document'] = test_data['document'].str.replace('[^가-힣 ]','')
test_data['document'] = test_data['document'].str.replace('^ +', '') #문단의 시작글자가 공백일 경우 지우라는 뜻
test_data['document'].replace('',np.nan, inplace=True)
test_data=test_data.dropna(how='any')
print(test_data.shape) #(143028, 3)

In [ ]:
# 불용어
stopwords = ['의','를','을','가','들','좀','잘','하여','으로','에','여','한','와','과','리','더','에는','하다','아','흠','별','막']

from tqdm import tqdm

okt=Okt()
x_train=[]
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence,stem=True)
    stop_remove_sentence = [word for word in tokenized_sentence if not word in stopwords] #불용어 제거
    x_train.append(stop_remove_sentence)

print(x_train[:3])


x_test=[]
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence,stem=True)
    stop_remove_sentence = [word for word in tokenized_sentence if not word in stopwords] #불용어 제거
    x_test.append(stop_remove_sentence)

print(x_test[:3])

tokenizer=Tokenizer()
tokenizer.fit_on_texts(x_train)
print(tokenizer.word_index)





In [ ]:
# 등장빈도가 3회 보다 적은 단어들 비중 확인
threshold = 3
total_cnt = len(tokenizer.word_index)
rare_cnt = 0  # 등장빈도가 3회 보다 적은 단어들 건수
total_freq = 0  # 전체 단어 빈도수 - 훈련 데이터에 대한
rare_freq = 0  # 등장빈도가 3회 보다 적은 단어들 비율

# 단어와 빈도수 k:v로 처리
for k, v in tokenizer.word_counts.items():
  total_freq = total_freq + v
  if v < threshold:
    rare_cnt = rare_cnt + 1
    rare_freq = rare_freq + v

print('total_cnt : ', total_cnt)
print('rare_cnt : ', rare_cnt)
print('3회 미만 단어 비율 : ', (rare_cnt / total_cnt) * 100)
print('전체 빈도에서 3회 미만 빈도 비율 : ', (rare_freq / total_freq) * 100)

# 전체 단어 수 중에서 빈도 수 2이하인 단어는 제거
vocab_size = total_cnt - rare_cnt + 1
print('vocab_size : ', vocab_size)

In [ ]:
# vocab_size : 19219
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
print(x_train[:3])

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

# empty smaple 제거 : 빈도수가 낮은 단어가 삭제됨으로 해서 빈도수가 낮은 단어로 구성된 샘플들은 empty sample이 됨
# 그러므로 제거작업이 필요
drop_train = [index for index, sentence in enumerate(x_train) if len(sentence) < 1]
print(drop_train)

x_train = np.delete(x_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(x_train), len(y_train))

# padding
print('리뷰 최대 길이 : ', max(len(review) for review in x_train))
print('리뷰 평균 길이 : ', sum(map(len, x_train)) / len(x_train))

max_len = 30  # padding의 요소 수는 대략 30을 부여
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

In [ ]:
# model
from keras.layers import LSTM, Embedding, Dense
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 100))  # 차원수는 100차원 줌
model.add(LSTM(128, activation='tanh'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

es = EarlyStopping(monitor='val_loss', mode='auto', patience=5)
mc = ModelCheckpoint('tf38.hdf5', monitor='val_loss', save_best_only=True)

history = model.fit(x_train, y_train, validation_split=0.2, batch_size=64, callbacks=[es, mc], verbose=2)

print('test 정확도 : ', model.evaluate(x_test, y_test)[1])
print('test 손실도 : ', model.evaluate(x_test, y_test)[0])

In [ ]:
mymodel = load_model('tf38.hdf5')

# 리뷰 예측
def predictFunc(new_sen):
  new_sen = re.sub(r'[^가-힣 ]', '', new_sen)  # 가~힣, 공백을 제거한 나머지만 남김
  new_sen = okt.morphs(new_sen, stem=True)  # 어간처리해줌
  new_sen = [word for word in new_sen if not word in stopwords]
  encoded = tokenizer.texts_to_sequences([new_sen])
  pad_new = pad_sequences(encoded, maxlen=max_len)
  score = float(mymodel.predict(pad_new))
  if score > 0.5:
    print('{:.2f}% 확률로 긍정'.format(score * 100))
  else:
    print('{:.2f}% 확률로 부정'.format(score * 100))

predictFunc('원작의 긴장감을 제대로 살려내지 못했다')
predictFunc('액션이 없는데도 재미 있는 몇안되는 영화')
predictFunc('보면서 웃지 않으면 환불')
predictFunc('엄청나게 재미없는 영화')
predictFunc('이 영화 맛도리다')